In [2]:
import pandas as pd
from sklearn import svm
from sklearn.metrics import classification_report,confusion_matrix,consensus_score
from sklearn.preprocessing import Normalizer

## Checkout dictionary

## Best Result 0.97

In [3]:
test_df = pd.read_csv("../../data/train_test_dataset/oilprice_cnbc_new_test.csv")
train_df = pd.read_csv("../../data/train_test_dataset/oilprice_cnbc_new_train.csv")

### 如果要測一不一樣的話

In [121]:
test_df_class = pd.read_csv("../../data/train_test_dataset/oilprice_cnbc_weird_test.csv")
test_df_class[test_df !=test_df_class].dropna(how="all")

In [69]:
# test_df['ans'] = 0
# test_df.loc[test_df['tags'].abs()>1,'ans']=1

In [4]:
x_drop = ['tags','date']
train_x = train_df.drop(x_drop,axis=1).values.astype(float)
train_y = train_df['tags'].values.astype(float)
test_x = test_df.drop(x_drop,axis=1).values.astype(float)
test_y = test_df['tags'].values.astype(float)

In [5]:
normalizer = Normalizer()
train_x = normalizer.fit_transform(train_x)
test_x = normalizer.transform(test_x)

In [6]:
# Perform classification with SVM, kernel=linear 
svc_model = svm.LinearSVC() 
svc_model.fit(train_x, train_y) 
prediction = svc_model.predict(test_x)


print (classification_report(test_y, prediction))

             precision    recall  f1-score   support

        0.0       1.00      0.99      0.99       417
        1.0       0.96      1.00      0.98       108

avg / total       0.99      0.99      0.99       525



In [8]:
print("TOTAL OF ",sum(test_y),"WITH ",sum(prediction),"1sssss")

TOTAL OF  108.0 WITH  113.0 1sssss


## Visualize Prediction

In [40]:
# plotly 
import plotly as py
import plotly.graph_objs as go
%load_ext autoreload
%autoreload 2

# offline plot  
# 因為如果寫成func去調用plotly會無法出現，所以只好用offline的方式。
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1. Visualize Test Data

In [41]:
effective_vs = test_df.loc[prediction==1]
non_effective_vs = test_df.loc[prediction!=1]
pairwise_dictionary = list(test_df.columns[1:-1])
trace1 = go.Bar(
    x=pairwise_dictionary,
    y=list(effective_vs.sum()[1:-1].values),
    name='Effective_sum'
)
trace2 = go.Bar(
    x=pairwise_dictionary,
    y=list(non_effective_vs.sum()[1:-1].values),
    name='Non_effective_sum'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='stacked-bar')

### 2. Visualize Train Data

In [51]:
effective_vs = train_df.loc[train_df.tags==1]
non_effective_vs = train_df.loc[train_df.tags!=1]
pairwise_dictionary = list(test_df.columns[1:-1])
trace1 = go.Bar(
    x=pairwise_dictionary,
    y=[ i/len(effective_vs) for i in list(effective_vs.sum()[1:-1].values)],
    name='Effective_sum'
)
trace2 = go.Bar(
    x=pairwise_dictionary,
    y=[ i/len(non_effective_vs) for i in list(non_effective_vs.sum()[1:-1].values)],
    name='Non_effective_sum'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='stacked-bar')